### Text to CLIP cross modality ###
This notebook is the implementation of our original hypothesis <br>
Before starting the open ended project, we have hypothesized that meaning of textual data (both individual words of concepts and full sentences) and images of the same concepts might be interpreted in the same places inside the brain. <br>
To try proving said hypothesis, we set out to expand on the work done by Pereira et al. (2018). In their work, "Toward a universal decoder of linguistic meaning from brain activation", Pereira et al. have made big strides in proving that meaning of different concepts, ranging from abstract to physical objects, is being parsed withing the brain in the same area. They scanned the

#### Setup ####

##### Dependecies #####
First, let's download all relevant dependecies to check our hypothesis

In [1]:
# Install dependencies
%pip install ftfy regex tqdm scikit-learn numpy matplotlib
%pip install -U gdown
%pip install git+https://github.com/openai/CLIP.git

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   -------------------------------------- - 7.9/8.1 MB 40.7 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 35.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 21.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ------------------------------- -------- 5.5/7.0 MB 25.8 MB/s eta 0:00:01
   ---------------------------------------- 7.0/7.0 MB 22.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\talgo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\talgo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/CLIP.git to c:\users\talgo\appdata\local\temp\pip-req-build-rxh8gjrr
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------------------- -------------------- 0.8/1.6 MB 8.5 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 10.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   - -------------------------------------- 8.4/241.3 MB 65.0 MB/s eta 0:00:04
   - -------------

  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\talgo\AppData\Local\Temp\pip-req-build-rxh8gjrr'

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\talgo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


##### Data #####
Now, we can import all of our relevant data! <br>
For this project, we've created a drive folder, containing all of the relevant code and data from the original Pereira et al. paper. Moreover, because the list of concepts and the related images from the original paper is static, we pre-calculated all of the relevant CLIP embeddings (The exact code we used can be seen here in "one_time_drive_setup"), and persisted them to drive. Let's download all of the relevant data, so we could continue our analysis. <br>

In [16]:
import platform
from pathlib import Path

# If the data already exists, we don't need to download it again
if not Path("data").exists():
    # Check operating system - handlind the data is done differently on Windows and Linux.
    # This will allow us to run the code on Colab, locally, and on any other platform we may choose.
    IS_WINDOWS = platform.system() == "Windows"
    
    if IS_WINDOWS:
        !python -m gdown --folder --id 1CwmFOsYFnq6t33KAzpvw0gaOTQXbcozs -O ./data/
        !powershell -NoProfile -Command "Expand-Archive -Path ./data/experiment-images.zip -DestinationPath ./data/ -Force"
        !powershell -NoProfile -Command "Remove-Item ./data/experiment-images.zip"
    else:
        !gdown --folder --id 1CwmFOsYFnq6t33KAzpvw0gaOTQXbcozs --output ./data
        !unzip ./data/experiment-images.zip
        !rm ./data/experiment-images.zip

Retrieving folder 19K2VYTfEe1pyp4I0r5b9BJ-R0lQNKQVe brain-responses-data
Processing file 1bIBosZxxCPhPvon3_ILDzYdVCVpRoUvR examples_180concepts_pictures.mat
Processing file 1lYl9lo2SuM6QI89evn1b5vFAi2kmY-XG examples_180concepts_sentences.mat
Processing file 1sFwuMcxYsO96I3mvMjUoHBpzqpulHkPK examples_180concepts_wordclouds.mat
Processing file 12DkhAFbWbjJ8lbf92r-V1M2IszFCycUI clip_image_embeddings.npz
Processing file 1GQbKpnB99I3InDqIo8PU_TT1Kk3GnizZ clip_text_embeddings.npz
Processing file 1KVmlIBNinqCzOYOQJyq4M5oceHRojqUk concepts.txt
Processing file 1i628lysbg60LZ9JN7l1cOq68GYWnxZ2S data_180concepts_sentences.mat
Processing file 1Z0EsEpmzHkuGcDh9e7KSQGK8RlUlrLbF experiment-images.zip


C:\Users\talgo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\gdown\__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Retrieving folder contents
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1bIBosZxxCPhPvon3_ILDzYdVCVpRoUvR
From (redirected): https://drive.google.com/uc?id=1bIBosZxxCPhPvon3_ILDzYdVCVpRoUvR&confirm=t&uuid=92c486e0-ac62-4da7-a9db-c480542f579d
To: c:\Users\talgo\OneDrive\Documents\Technion baby\2025ב\שפה\fmri-CLIP-decoder\data\brain-responses-data\examples_180concepts_pictures.mat

  0%|          | 0.00/287M [00:00<?, ?B/s]
  0%|          | 524k/287M [00:00<01:12, 3.93MB/s]
  1%|          | 2.10M/287M [00:00<00:31, 8.93MB/s]
  2%|▏         | 6.29M/287M 

For the embeddings, we saved the following:
* <u>Textual data</u> - we saved the embedding of the prompt "A picture of {c}", where c is the name of the relevant concept. The reason for that choice is that CLIP reacts very well to prompting, and that embedding is improving results over the non-prompt version.
* <u>Visual data</u> - because each concept has 6 separate images describing it, we embedded all of them to CLIP's embedding space. Since this is not a singular vector, there are several approaches on how to parse the data, but the base embedding will be the same nonetheless, so we saved the embeddings for all images.

In [7]:
import numpy as np

with np.load("data/clip_text_embeddings.npz") as text_embeddings:
    clip_text_embeddings = text_embeddings["data"]

with np.load("data/clip_image_embeddings.npz") as image_embeddings:
    clip_image_embeddings = image_embeddings["embeddings"]

#### Training ####
As stated earlier, we are going to keep Pereira et al's method of learning a **linear** decoder, that would be able to generalize from the data it saw to unseen data, and thus capture deeper meaning than the training data itself. In the original paper, they managed to achieve results which are **much** better than chances, implying that a linear decoder is more than sufficient to capture meaning of textual-fMRI data when the data is projected onto an embedding space. <br>
We have changed the embedding space from GloVe to CLIP, in order to see if textual-fMRI data can capture meaning of images as well, but we believe that adding extra complexity to the model will defeat our purpose. If our hypothesis is correct, then a linear decoder will be able to capture the meaning of the images in the same embedding space, without adding any non-linearity - a simple model should suffice, as much as is did for the same modality data. <br>
For that reason, we are going to train our decoder in the same way that it was trained in the original paper:
* For each participant's fMRI data - we're only going to take the top 5000 relevant `voxels`.
* For the decoder - we're going to learn a simple ridge regression (using the same code)
<br> 
Another important thing to remember is that we're only going to feed the function **textual** fMRI data, because our hypothesis states that from the same areas in the brain responsible for interpreting textual data can also give us insight on visual data. That means that we will only train our model on textual data, and withold any visual data for evaluation only.

First, let's define the function that will help us determine the top 5000 relevant voxels: <br>
The fMRI data from the experiments consists of a big series of voxel each corresponding to the activation in a different area in the brain. The problem here is that most of the voxels are non-importent and are just noise which will reduce our model's accuracy. For that reason we will clean up the data and only use the 5000 most influencing voxels out of the 200,000 in the original fMRI data and we will be doing so using the select_top_voxels_indexes function:

In [8]:
from sklearn.feature_selection import f_regression
import numpy as np

def select_top_voxels_indices(fmri_data, semantic_vectors, num_voxels=5000):
    f_scores = []
    for i in range(semantic_vectors.shape[1]):
        f, _ = f_regression(fmri_data, semantic_vectors[:, i])
        f_scores.append(f)

    f_scores = np.array(f_scores)
    voxel_scores = np.max(f_scores, axis=0)
    top_voxel_indices = np.argsort(voxel_scores)[-num_voxels:]

    return top_voxel_indices

Now, let's take the fMRI textual data from our data folder:

In [9]:
import scipy.io

mat = scipy.io.loadmat("data/brain-responses-data/examples_180concepts_wordclouds.mat")
fmri_text_data = mat["examples"]

and get only our top voxels:

In [10]:
top_voxel_indices = select_top_voxels_indices(fmri_text_data, clip_text_embeddings)
reduced_fmri_data = fmri_text_data[:, top_voxel_indices]

Finally, let's train our textual decoder. We'll use the function "learn_decoder", which we took directly from the original paper:

In [11]:
""" learn_decoder """
import sklearn.linear_model

def learn_decoder(data, vectors):
     """ Given data (a CxV matrix of V voxel activations per C concepts)
     and vectors (a CxD matrix of D semantic dimensions per C concepts)
     find a matrix M such that the dot product of M and a V-dimensional 
     data vector gives a D-dimensional decoded semantic vector. 

     The matrix M is learned using ridge regression:
     https://en.wikipedia.org/wiki/Tikhonov_regularization
     """
     ridge = sklearn.linear_model.RidgeCV(
         alphas=[1, 10, .01, 100, .001, 1000, .0001, 10000, .00001, 100000, .000001, 1000000],
         fit_intercept=False
     )
     ridge.fit(data, vectors)
     return ridge.coef_.T

The training itself:

In [12]:
decoder = learn_decoder(reduced_fmri_data, clip_text_embeddings)

#### Evaluation
For evaluation, we're going to have the following guiding principles:
1. "Distance" between datapoints - 


1. Strategy - because quality fMRI data is extremely limited (Pereira is still the only open English dataset for fMRI single conept data, similary to what we're trying to model. There are some other alternatives - Allen 672 which is Chinese, Tuckute 2024, which is for full sentences only, six words each, each of them on no more than 16 participants), we don't have much data to work with. That means we want to base our decoder on all of the available data, scarce as it is. Because of that fact, we'll choose to evaluate the data using K-Fold Cross Validation - that way, we can train on every piece of data we have, and still evaluate the model.

### Multimodal to Cross-Modality


To attempt to disprove our initial hypothesis we will attempt to train a decoder on both the concepts and images fmri data and test it on the image embedding, it should have a much higher accuracy 

#### Load Core Data (fMRI, concepts, CLIP targets)  
First we load `concepts_fmri_data` / `pictures_fmri_data`: the fmri data from the first experiment.  

In [ ]:
import scipy.io

def extract_data_from_mat_file(path):
    return scipy.io.loadmat(path)["examples"]

concepts_fmri_data = extract_data_from_mat_file("data/brain-responses-data/examples_180concepts_wordclouds.mat")
pictures_fmri_data = extract_data_from_mat_file("data/brain-responses-data/examples_180concepts_wordclouds.mat")

#### Per-Concept Image Embedding Averages  
Aggregates all image embeddings for each concept (~6 images) into a single CLIP vector by averaging and L2-normalizing.  

In [ ]:
import os
import numpy as np

def compute_average_clip_embeddings(images_dir, clip_image_embeddings):
    """
    Compute the average CLIP image embedding for each concept folder.
    """
    
    concept_folders = sorted(os.listdir(images_dir))

    averaged_embeddings = []
    embedding_index = 0  # pointer into clip_image_embeddings

    for concept in concept_folders:
        folder_path = os.path.join(images_dir, concept)

        # Get all .jpg images in this concept folder
        image_files = sorted([
            f for f in os.listdir(folder_path)
            if f.lower().endswith(".jpg")
        ])

        num_images = len(image_files)
        if num_images == 0:
            print(f"⚠️ No images found for '{concept}', skipping.")
            continue

        # Get the embeddings of the images for the concept
        concept_embeds = clip_image_embeddings[
            embedding_index : embedding_index + num_images
        ]
        embedding_index += num_images

        # Average of embeddings
        avg_embed = concept_embeds.mean(axis=0)

        # Normalization
        norm = np.linalg.norm(avg_embed)
        if norm > 0:
            avg_embed /= norm

        averaged_embeddings.append(avg_embed)

    averaged_embeddings = np.array(averaged_embeddings, dtype=np.float32)
    print("Averaged embeddings shape:", averaged_embeddings.shape)

    return averaged_embeddings

In [34]:
images_dir = "data/experiment-images"  # path to your images
clip_image_embedding_averages = compute_average_clip_embeddings(
    images_dir,
    clip_image_embeddings
)

Averaged embeddings shape: (180, 768)


#### Voxel Selection (Top-K per Modality)  
Ranks voxels by max absolute correlation with CLIP targets:  

- For concepts: fMRI (concepts) vs text embeddings.  
- For pictures: fMRI (pictures) vs image averages.  

Takes the top 5,000 indices from each list.  
Creates two candidate voxel sets tuned to each modality.  

In [27]:
top_voxels_num = 5000  # top voxels per dataset

idx_concepts  = select_top_voxels_indices(concepts_fmri_data,  clip_text_embeddings, top_voxels_num)
idx_pictures  = select_top_voxels_indices(pictures_fmri_data,  clip_image_embedding_averages, top_voxels_num)

print(len(idx_concepts), len(idx_pictures))

5000 5000


#### Union of Voxel Sets (Collision-Aware)  
Builds a union of voxel indices from both lists, preserving first appearance.  
If a voxel appears in both lists (collision), averages the two fMRI columns:  

- unique to concepts → `concepts_fmri_data[:, v]`  
- unique to pictures → `pictures_fmri_data[:, v]`  
- in both → `0.5 * (concepts[:, v] + pictures[:, v])`  

Yields `combined_fmri_data` with shape (180, K_unique). 

In [ ]:
import numpy as np

def build_union_fmri_matrix(
    idx_concepts: np.ndarray,
    idx_pictures: np.ndarray,
    concepts_fmri_data: np.ndarray,
    pictures_fmri_data: np.ndarray,
) -> tuple[np.ndarray, np.ndarray]:
    """
    Merge voxel indices from concepts and pictures, preserving first-seen order.
    For indices present in BOTH lists, average their voxel columns from the two fMRI matrices.
    For indices present in only one list, take that source's voxel column.
    """
    # Basic sanity checks
    assert concepts_fmri_data.shape[0] == pictures_fmri_data.shape[0], \
        "Row mismatch: concepts_fmri_data and pictures_fmri_data must have same N (items)."
    N, Vc = concepts_fmri_data.shape
    _, Vp = pictures_fmri_data.shape
    assert Vc == Vp, "Voxel count mismatch: concepts and pictures must have same number of voxel columns."

    # Normalize inputs to int arrays
    idx_concepts = np.asarray(idx_concepts, dtype=int)
    idx_pictures = np.asarray(idx_pictures, dtype=int)

    # Build ordered union and record sources for each voxel
    sources_by_voxel: dict[int, set[str]] = {}
    union_indices_list: list[int] = []

    def _add_indices(indices, tag):
        for v in indices:
            v = int(v)
            if v not in sources_by_voxel:
                sources_by_voxel[v] = {tag}
                union_indices_list.append(v)   # preserve first-seen order
            else:
                sources_by_voxel[v].add(tag)

    _add_indices(idx_concepts, "concepts")
    _add_indices(idx_pictures, "pictures")

    union_indices = np.array(union_indices_list, dtype=int)
    print(f"Union voxel count: {union_indices.size}")

    # Build combined matrix with collision handling
    combined_fmri_data = np.empty((N, union_indices.size), dtype=np.float32)

    for j, v in enumerate(union_indices):
        srcs = sources_by_voxel[v]
        if "concepts" in srcs and "pictures" in srcs:
            # Collision → average the columns from both sources
            combined_fmri_data[:, j] = 0.5 * (concepts_fmri_data[:, v] + pictures_fmri_data[:, v])
        elif "concepts" in srcs:
            combined_fmri_data[:, j] = concepts_fmri_data[:, v]
        else:
            combined_fmri_data[:, j] = pictures_fmri_data[:, v]

    print("combined fmri data shape:", combined_fmri_data.shape)
    return union_indices, combined_fmri_data

In [36]:
union_indices, combined_fmri_data = build_union_fmri_matrix(
    idx_concepts=idx_concepts,
    idx_pictures=idx_pictures,
    concepts_fmri_data=concepts_fmri_data,
    pictures_fmri_data=pictures_fmri_data,
)

Union voxel count: 9740
combined fmri data shape: (180, 9740)


#### Train Decoder: fMRI → CLIP Text Space  
Trains a decoder mapping `combined_fmri_data` → CLIP text embeddings.   

In [ ]:
combined_decoder = learn_decoder(combined_fmri_data, clip_text_embeddings)